### Summary

Refactor the `linear_solvers` module, so that the quantum algorithm (HHL) accepts oracles for the different blocks and computes different parameters according to the error bounds calculated in [1].

### Workflow

An instance of the algorithm can be created by specifying the error tolerance, the problem to be solved is only specified when the `solve()` method is called:
```python
from qiskit.aqua.algorithms.linear_solvers.hhl import HHL

# initialise the algorithm. HHL() defaults to epsilon=1e-2
hhl = HHL(epsilon=1e-3, further_params=...)
```

Alternatively, it is possible to specify the different parameters of HHL (i.e. the number of qubits to represent the eigenvalues, the evolution time, ...) through setter methods. However it should be noted that in this case the error tolerance won't be guaranteed anymore. The reason for this is because from the theory in [1] we can calculate the different parameters to achieve an overall accuracy `epsilon`, but once one of these parameters is changed by the user, `epsilon` will change as well and is difficult to compute, so should be set to `None` (should this give a warning?).

```python
hhl.n_l = 5 # number of qubits to represent the eigenvalues. Set epsilon=None and possibly raises a warning.
```

The `solve()` method accepts up to five arguments: 
```python
def solve(self, matrix: Union[np.ndarray, QuantumCircuit],
          vector: Union[np.ndarray, QuantumCircuit],
          observable: Optional[Union[LinearSystemObservable, BaseOperator,
                                     List[BaseOperator]]] = None,
          post_rotation: Optional[Union[QuantumCircuit, List[QuantumCircuit]]] = None,
          post_processing: Optional[Callable[[Union[float, List[float]]],
                                             Union[float, List[float]]]] = None) \
        -> LinearSolverResult:
```
The first two are the matrix defining the linear system and the vector right hand side of the equation (i.e. writing $Ax=b$, respectively $A$ and $b$). The remaining parameters concern the (list of) observable(s) to be computed out of the solution vector $x$, and can be specified in two different ways. One option is to give as the third and last parameter a (list of) `LinearSystemObservable`(s). Alternatively, we can give our own implementations of the `observable`, `post_rotation` and `post_processing`, where
- `observable` is the operator to compute the expected value of the observable and can be e.g. a `PauliSumOp`
- `post_rotation` is the circuit to be applied to the solution to extract information if additional gates are needed.
- `post_processing` is the function to compute the value of the observable from the calculated probabilities.

Observables are required because the output of the quantum algorithm is a quantum state representing the vector $x$, however to learn all the components of this vector would take a linear time in its dimension diminishing any speedup obtained by the quantum algorithm. Thus, we can only compute functions from $x$ (the so called observables) to learn information about the solution.

#### Input matrix

There are several ways to specify $A$:
- Most intuitive, although only useful for illustration, it can be specified as a numpy array:

```python
matrix = [[1/2, -2], [-2, 1/2]]
```

- If we know how to implement $e^{iAt}$, we can also give this circuit as an input. The circuit needs to be able to accept $t$ as a parameter, since this parameter will be specified within HHL and will change during the run of the algorithm:

```python
from qiskit.circuit.library.blueprintcircuit import BlueprintCircuit
class EvolutionCircuit(BlueprintCircuit):
    def __init__(self, num_state_qubits, name='mcirc'):
        super().__init__(name=name)
        self.num_state_qubits = num_state_qubits
        
    # ...
    
    def _build(self, evolution_time=1):  # should we introduce an interface for such circuits?
        # build code
        
matrix = EvolutionCircuit(1)
```

- The matrix can be given as an operator (which can be evolved):

```python
matrix = (-1.05 * I) + (0.39 * Z) + (0.18 *  X) 

```

- The matrix can be given by a `time_evolution` algorithm, which needs to be added late as it is not defined yet.




#### Input vector

HHL also accepts different types for the vector. Again, it can be specified as a numpy array for illustration:
```python
vector = [1, 0]
```
or as a circuit
```python
from qiskit import QuantumCircuit

vector = QuantumCircuit(1)
vector.h(0)
```

#### Input observable

There are several types of available observables which can be given as input:

```python
from qiskit.aqua.algorithms.linear_solvers.observables import AbsoluteAverage, ExpectedValue, MatrixFunctional

observables = [AbsoluteAverage(...), ExpectedValue(...)]
hhl = HHL(...)
```

The `MatrixFunctional` class takes the main and off diagonal values of a tridiagonal symmetric Toeplitz matrix for its constuctor method.

```python
observables.append(MatrixFunctional(1, 1/2))
```

Once these inputs are specified, we can run the algorithm and obtain the calculated value for each observable:
However some observables will depend on the dimension of the system, which ideally one only specifies once `hhl.solve` is called. Therefore, we could allow to add observables from this method as well:

```python
# run the algorithm and add an observable which depends on the dimension 
result = hhl.solve(matrix=matrix, vector=vector, observable=observables)

# array with the calculated values for each observable
observable_values = result.observable

```

Further, the linear solver result contains a `result.state` method which returns the quantum circuit that prepares the solution state:
```python
solution = result.state

```

Finally, some algorithms can calculate the Euclidean norm of the vector solution. This can be accessed via
```python
result.euclidean_norm
```
which returns either a `float` or `None`.

--------------------  This is not supported yet -----------------------------------------------------------------------

Another option is to run the `LinearSystem` algorithm without specifying an observable and then use the circuit as the input for the observable class:

```python
hhl = HHL(epsilon=1e-3)
circuit = hhl.construct_circuit(matrix, vector, uncompute=True)  # allows some options
# or 
result = hhl.solve(matrix, vector)
circuit = result.solution

observable = AbsoluteAverage()

result = observable.evaluate(circuit)  # can also evaluate arrays, statevectors.
```
---------------------- Until here--------------------------------------------------------------------------------------
#### State and working qubits

Many circuits require working qubits. To distinguish between working and state qubits we use the qiskit `ancilla`-qubit flag. All non-flagged qubits are considered as state qubits.

#### Repeat-until-success (RUS) circuits

It is quite common, e.g., for state preparation, i.e., preparing the vector, that circuits have to be repeated until some success criteria is met.
Since this concept is not explicitely supported by Qiskit yet, we consider each circuit with measurements as a RUS circuit and will assume it has been successful if all measurements return state $|1\rangle$.

Question: how should we handle this in the statevector simulation?



### Future work

- Add the Richardson extrapolation logic.
- Add a `StateObservable` which returns the full vector solution (e.g. tomography).

### References
[1] Carrera Vazquez, A., Hiptmair, R., & Woerner, S. (2020). Enhancing the Quantum Linear Systems Algorithm using Richardson Extrapolation. `arXiv:2009.04484 <http://arxiv.org/abs/2009.04484>`

# Current working version example

```python
from qiskit.aqua.algorithms.linear_solvers_new.observables.absolute_average import AbsoluteAverage
from qiskit.aqua.algorithms.linear_solvers_new.hhl import HHL
from qiskit.aqua.algorithms.linear_solvers_new.tridiagonal import Tridiagonal
from scipy.sparse import diags
from qiskit import QuantumCircuit, quantum_info
from qiskit import Aer, execute
import numpy as np
from qiskit.aqua.algorithms.linear_solvers_new.observables.matrix_functional import MatrixFunctional

mat_func = MatrixFunctional(1,1/2)

a = 1
b = 1/3
nb = 2
qc = QuantumCircuit(nb)
# qc.x(0)
# qc.h(0)
ham = Tridiagonal(nb,a,b,trotter=2)
hhl = HHL()
# solution = hhl.solve(ham,qc,AbsoluteAverage())
solution = hhl.solve(ham,qc,mat_func)

# Analytic solution
matrix = diags([b, a, b], [-1, 0, 1], shape=(2**nb,2**nb)).toarray()
b = np.array([1]+[0]*(2**nb -1))
x = np.dot(np.linalg.inv(matrix), b)

x/np.linalg.norm(x)
solution.euclidean_norm
# Value of the observable
solution.observable
# Analytic value of the observable
mat_func.numpy_solver(x)
#AbsoluteAverage.numpy_solver(x)

```